In [1]:
import numpy as np
import pandas as pd
import os
import math
import random
import csv
import sys
from sklearn import metrics
from sklearn.metrics import classification_report

from bert_sklearn import BertClassifier
from bert_sklearn import BertRegressor
from bert_sklearn import load_model

DATADIR = os.getcwd() + '/glue_examples/glue_data'

def read_tsv(filename,quotechar=None):
    with open(filename, "r", encoding='utf-8') as f:
        return list(csv.reader(f,delimiter="\t",quotechar=quotechar))   

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.



We will use the SST-2 (Stanford Sentiment Treebank) data set.

The input features are short sentences and the labels are the standard sentiment polarity of:
*    0 for negative 
*    1 for positive.

In [2]:
%%bash
python3 ./glue_examples/download_glue_data.py --data_dir ./glue_examples//glue_data --tasks SST 

	Completed!


In [2]:
"""
SST-2 train data size: 67349 
SST-2 dev data size: 872 
"""
def get_sst_data(train_file = DATADIR + '/SST-2/train.tsv',
                dev_file  = DATADIR + '/SST-2/dev.tsv'):
    
    train = pd.read_csv(train_file, sep='\t',  encoding = 'utf8',keep_default_na=False)
    train.columns=['text','label']
    print("SST-2 train data size: %d "%(len(train)))
    
    dev = pd.read_csv(dev_file, sep='\t',  encoding = 'utf8',keep_default_na=False)
    dev.columns=['text','label']
    print("SST-2 dev data size: %d "%(len(dev)))
    label_list = np.unique(train['label'])
    
    return train,dev,label_list

train,dev,label_list = get_sst_data()

# subsample data for demo
train = train.sample(1000,random_state=42)

X_train = train['text']
y_train = train['label']

X_dev = dev['text']
y_dev = dev['label']

train.head()

SST-2 train data size: 67349 
SST-2 dev data size: 872 


,text,label
66730,with outtakes in which most of the characters ...,0
29890,enigma is well-made,1
45801,is ) so stoked to make an important film about...,0
29352,the closest thing to the experience of space t...,1
19858,lose their luster,0


Suppose we want to tune over some the hyperparameters mentioned in the paper:
* **`epochs`** in  [3, 4]
* **`learning rate`** in  [2e-5, 3e-5, 5e-5]

## Option 1: Use **`tune_params`** to  fit on train and tune  on holdout set 

Here the variation can come from different random states used in the fitting

The default in **`tune_params`** has 4 random  seeds

In [4]:
%%time

# define model
model = BertClassifier()

# tune model
tuned = model.tune_params(X_train, y_train,
                          X_dev, y_dev,
                          epochs = [3,4],                          
                          learning_rate = [2e-5, 3e-5, 5e-5])


Building sklearn classifier...
epochs: 3
learning_rate: 2e-05
max_seq_length: 128
num_mlp_layers: 0
train_batch_size: 32
Using random seed : 42
Building sklearn classifier...
Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
train data size: 1000, validation data size: 0


Training: 100%|██████████| 32/32 [00:23<00:00,  1.53it/s, loss=0.15] 



Test loss: 0.3114, Test accuracy = 87.73%
score: 87.73

Using random seed : 134
Building sklearn classifier...
Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
train data size: 1000, validation data size: 0


Training: 100%|██████████| 32/32 [00:24<00:00,  1.47it/s, loss=0.163]



Test loss: 0.3054, Test accuracy = 87.96%
score: 87.96

Using random seed : 6
Building sklearn classifier...
Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
train data size: 1000, validation data size: 0


...


Test loss: 0.3685, Test accuracy = 87.61%
score: 87.61

Using random seed : 8
Building sklearn classifier...
Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
train data size: 1000, validation data size: 0


Training: 100%|██████████| 32/32 [00:24<00:00,  1.45it/s, loss=0.0409]



Test loss: 0.3433, Test accuracy = 88.65%
score: 88.65

87.987 (+/-1.098) for {'epochs': 3, 'learning_rate': 2e-05, 'max_seq_length': 128, 'num_mlp_layers': 0, 'train_batch_size': 32}
87.787 (+/-1.896) for {'epochs': 3, 'learning_rate': 3e-05, 'max_seq_length': 128, 'num_mlp_layers': 0, 'train_batch_size': 32}
86.898 (+/-1.967) for {'epochs': 3, 'learning_rate': 5e-05, 'max_seq_length': 128, 'num_mlp_layers': 0, 'train_batch_size': 32}
87.615 (+/-1.228) for {'epochs': 4, 'learning_rate': 2e-05, 'max_seq_length': 128, 'num_mlp_layers': 0, 'train_batch_size': 32}
88.045 (+/-1.790) for {'epochs': 4, 'learning_rate': 3e-05, 'max_seq_length': 128, 'num_mlp_layers': 0, 'train_batch_size': 32}
87.385 (+/-3.025) for {'epochs': 4, 'learning_rate': 5e-05, 'max_seq_length': 128, 'num_mlp_layers': 0, 'train_batch_size': 32}
Best mean score is 88.04, with params: {'epochs': 4, 'learning_rate': 3e-05, 'max_seq_length': 128, 'num_mlp_layers': 0, 'train_batch_size': 32}
Building sklearn classifier...

Training: 100%|██████████| 32/32 [00:24<00:00,  1.46it/s, loss=0.0569]

CPU times: user 37min 26s, sys: 17min 57s, total: 55min 23s
Wall time: 44min 44s


In [5]:
# scores
all_scores = tuned['scores']

best_param = tuned['best_param']

best_score = tuned['best_score']

best_model = tuned['best_model']

best_model, best_param, best_score

(BertClassifier(bert_model='bert-base-uncased', epochs=4, eval_batch_size=8,
         fp16=False, gradient_accumulation_steps=1,
         label_list=array([0, 1]), learning_rate=3e-05, local_rank=-1,
         logfile='bert_sklearn.log', loss_scale=0, max_seq_length=128,
         num_mlp_hiddens=500, num_mlp_layers=0, random_state=6,
         restore_file=None, train_batch_size=32, use_cuda=True,
         validation_fraction=0, warmup_proportion=0.1),
 {'epochs': 4,
  'learning_rate': 3e-05,
  'max_seq_length': 128,
  'num_mlp_layers': 0,
  'train_batch_size': 32},
 88.0447247706422)

## Option 2:  Use sklearn's **`GridSearchCV`** to to fit on cv splits of train data

In [5]:
%%time
from sklearn.model_selection import GridSearchCV

params = {'epochs':[3, 4], 'learning_rate':[2e-5, 3e-5, 5e-5]}

# wrap classifier/regressor in GridSearchCV
clf = GridSearchCV(BertClassifier(validation_fraction=0), 
                    params,
                    scoring='accuracy',
                    verbose=True)

# fit gridsearch 
clf.fit(X_train ,y_train)

Building sklearn classifier...
Fitting 3 folds for each of 6 candidates, totalling 18 fits
Building sklearn classifier...
Building sklearn classifier...
Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
train data size: 666, validation data size: 0


Training: 100%|██████████| 21/21 [00:15<00:00,  1.41it/s, loss=0.127]


Building sklearn classifier...
Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
train data size: 667, validation data size: 0


Training: 100%|██████████| 21/21 [00:15<00:00,  1.34it/s, loss=0.118]


Building sklearn classifier...
Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
train data size: 667, validation data size: 0


...

Building sklearn classifier...
Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
train data size: 667, validation data size: 0


Training: 100%|██████████| 21/21 [00:16<00:00,  1.33it/s, loss=0.146]
[Parallel(n_jobs=1)]: Done  18 out of  18 | elapsed: 24.2min finished


Building sklearn classifier...
Loading bert-base-uncased model...
Defaulting to linear classifier/regressor
train data size: 1000, validation data size: 0


Training: 100%|██████████| 32/32 [00:24<00:00,  1.45it/s, loss=0.15] 

CPU times: user 21min 16s, sys: 9min 59s, total: 31min 16s
Wall time: 25min 32s


In [6]:
# best scores
print(clf.best_score_, clf.best_params_)

means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']

for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))

0.891 {'epochs': 3, 'learning_rate': 2e-05}
0.891 (+/-0.024) for {'epochs': 3, 'learning_rate': 2e-05}
0.882 (+/-0.010) for {'epochs': 3, 'learning_rate': 3e-05}
0.814 (+/-0.089) for {'epochs': 3, 'learning_rate': 5e-05}
0.877 (+/-0.005) for {'epochs': 4, 'learning_rate': 2e-05}
0.885 (+/-0.022) for {'epochs': 4, 'learning_rate': 3e-05}
0.883 (+/-0.010) for {'epochs': 4, 'learning_rate': 5e-05}
